# 2. Netron-Torch网络结构可视化
介绍一种可视化Torch网络结构的应用[Netron](https://github.com/lutzroeder/netron)，其支持读取Torch输出的.onnx格式模型文件，并在应用中作出模型结构图。

这里给出一个应用案例，我们将1-TorchModel.ipynb中的VAE网络结构可视化：

In [15]:
import torch
from torch import nn, optim
from torch.nn import functional as F

# Model Definition
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20) # Mu
        self.fc22 = nn.Linear(400, 20) # log(Var)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)
    
    def encoder(self, x):
        h1 = F.relu(self.fc1(x))
        mu = self.fc21(h1)
        logvar = self.fc22(h1)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))
    
    def forward(self, x):
        y = x.view(-1, 784)
        mu, logvar = self.encoder(y)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Model
vae = VAE()

# Input
input = torch.randn([1, 1, 28, 28])

# Save Model and Input to onnx file
torch.onnx.export(vae, input, "./data/models/VAE.onnx")

利用Netron应用打开模型后，得到模型结构如图所示：

<p align=center>
<img src="./fig/2-1.png" width=1500>
</p>

其中，还可以单击各个模块查看变换细节：

<p align=center>
<img src="./fig/2-2.png" width=1000>
</p>